### Initialization

In [ ]:
import numpy as np
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
def get_EU(X, p, alpha):
    return p*(X**alpha)

def get_pk(X_ref, p_ref, X_lot, p_lot, alpha, beta):
    # get the probability of choosing lotery, from a set of X, p, and alpha and beta parameters
    EUr = get_EU(X_ref, p_ref, alpha)
    EUl = get_EU(X_lot, p_lot, alpha)
    # changed the minus sign inside the exponent.
    # standard logistic model, and simplify expressions (same result)
    return 1/(1 + np.exp(-beta*(EUl - EUr))), EUr, EUl

def get_choices(X_ref, p_ref, X_lot, p_lot, alpha, beta):
    pk, EUr, EUl = get_pk(X_ref, p_ref, X_lot, p_lot, alpha, beta)
    choices = np.random.uniform(size=EUl.shape)<pk
    return choices, pk, EUr, EUl

def get_nll_standard(X_ref, p_ref, X_lot, p_lot, choices, alpha, beta):
    ''' compute neg-log-likelihood for a single experiment
    inputs are:
      - the parameters of the task (values and probabilities of lottery and reference)
      - the subject's choices
      - a guess on the subject parameteres (alpha and beta)
    It uses the whole formula, which might be "numerically unstable" because of divisions by 0
    '''
    EUr = p_ref*X_ref**alpha
    EUl = p_lot*X_lot**alpha
    chose_ref = choices==False
    ll_v = 1/(1 + np.exp(-beta*(EUl - EUr)))
    ll_v[chose_ref] = 1 - ll_v[chose_ref]
    return -np.sum(np.log(ll_v))

def get_nll(X_ref, p_ref, X_lot, p_lot, choices, alpha, beta):
    ''' compute neg-log-likelihood for a single experiment
    inputs are:
      - the parameters of the task (values and probabilities of lottery and reference)
      - the subject's choices
      - a guess on the subject parameteres (alpha and beta)
    
    Here we don't compute the likelihood with the whole logistic formula: pk = 1/(1+exp(-beta*(EUl-EUr))
    We can use logarithmic properties to get rid of divisions (which I think is the point of using the -log-likelihood instead of the likelihood)
    For the pos. choices, we use log(1/whatever) = log(1) - log(whatever) = -log(whatever)
    For the neg. choices, we use log(1 - 1/(1+exp(y))) = log(exp(y)/(1+exp(y))) = y - log(1+exp(y))
    Which is y + (thing_with_pos_choice)
    As there are no divisions, no div/0 expected, but there still might be infinites (overflow) ¯\_(ツ)_/¯
    '''
    # probably this is not complex enough to need its own function, save time in function calling
    EUr = p_ref*X_ref**alpha 
    EUl = p_lot*X_lot**alpha

    # Compute things once, sign flip to possibly save one operation
    y = beta*(EUr - EUl)
    chose_ref = choices==False

    # values to be summed up, already negative:
    nll_v = np.log(1 + np.exp(y))
    nll_v[chose_ref] = nll_v[chose_ref] - y[chose_ref]
    return np.sum(nll_v)


def get_nll_clean(X_ref, p_ref, X_lot, p_lot, choices, alpha, beta):
    ''' compute neg-log-likelihood for a single experiment
    inputs are:
      - the parameters of the task (values and probabilities of lottery and reference)
      - the subject's choices
      - a guess on the subject parameteres (alpha and beta)
    
    Here we don't compute the likelihood with the whole logistic formula: pk = 1/(1+exp(-beta*(EUl-EUr))
    We can use logarithmic properties to get rid of divisions (which I think is the point of using the -log-likelihood instead of the likelihood)
    For the pos. choices, we use log(1/whatever) = log(1) - log(whatever) = -log(whatever)
    For the neg. choices, we use log(1 - 1/(1+exp(y))) = log(exp(y)/(1+exp(y))) = y - log(1+exp(y))
    Which is y + (thing_with_pos_choice)
    As there are no divisions, no div/0 expected, but there still might be infinites (overflow) ¯\_(ツ)_/¯
    '''
    # probably this is not complex enough to need its own function, save time in function calling
    EUr = p_ref*X_ref**alpha 
    EUl = p_lot*X_lot**alpha

    # Compute things once, sign flip to possibly save one operation
    y = beta*(EUr - EUl)
    chose_ref = choices==False

    # values to be summed up, already negative:
    nll_v = np.log(1 + np.exp(y))
    nll_v[chose_ref] = nll_v[chose_ref] - y[chose_ref]
    return np.sum(nll_v[np.isfinite(nll_v)])

def optimize_brute_force(X_ref, p_ref, X_lot, p_lot, choices, alpha_range, beta_range, steps, get_nll_fun=get_nll):
    ''' compute -log-likelihood alpha/beta values within a range and with a resolution
    return the alpha, beta and -log-likelihood grids
    '''
    alpha_vals = np.linspace(alpha_range[0], alpha_range[1], steps)
    beta_vals = np.linspace(beta_range[0], beta_range[1], steps)

    (alpha_grid, beta_grid) = np.meshgrid(alpha_vals, beta_vals)
    alphabeta_shape = alpha_grid.shape
    alpha_grid = alpha_grid.flatten()
    beta_grid = beta_grid.flatten()

    nll = [get_nll_fun(X_ref, p_ref, X_lot, p_lot, choices, alpha_grid[_], beta_grid[_]) 
           for _ in np.arange(len(alpha_grid))]
    alpha_grid = alpha_grid.reshape(alphabeta_shape)    
    beta_grid = beta_grid.reshape(alphabeta_shape)    
    nll = np.asarray(nll).reshape(alphabeta_shape)
    
    return alpha_grid, beta_grid, nll


### Test on stability with a single "subject"

In [ ]:
# subject parameters
alpha = 0.7
beta = 0.8

# task parameters
X_ref = 20
p_ref = 1
X_vals = [20, 30, 40, 70, 100]
p_vals = [.1, .25, .5, .75, .9]
trialsPerComb = 6

# get all combinations for trials
(X_grid, p_grid) = np.meshgrid(X_vals, p_vals)
X_cmb_unique = np.tile(X_grid.flatten(), trialsPerComb)
p_cmb_unique = np.tile(p_grid.flatten(), trialsPerComb)
X_lot_trials = np.tile(X_cmb_unique, trialsPerComb)
p_lot_trials = np.tile(p_cmb_unique, trialsPerComb)


choices, pk, EUr, EUl = get_choices(X_ref, p_ref, X_lot_trials, p_lot_trials, alpha, beta)


# compare several nll functions:
funs = (get_nll_standard, get_nll, get_nll_clean)
fig = make_subplots(rows=1, cols=len(funs), subplot_titles=[_.__name__ for _ in funs])

for k_fun in np.arange(len(funs)):
    nll_fun = funs[k_fun]
    print(f"Using function {nll_fun.__name__}...")
    print('  computing...')
    start_time = time.time()
    nll = nll_fun(X_ref, p_ref, X_lot_trials, p_lot_trials, choices, alpha, beta)
    end_time = time.time()
    el_time = end_time-start_time
    print(f'    {nll_fun.__name__} took {el_time} s')
    print(f'    Result: {nll}')

    # test numerical stability
    print('testing stability...')
    alpha_range = (0.1, 2)
    beta_range = (-4, 4)
    steps = 100
    print('  computing brute force...')
    alpha_grid, beta_grid, nll_map = optimize_brute_force(X_ref, p_ref, X_lot_trials, p_lot_trials, choices, alpha_range, beta_range, steps, get_nll_fun=nll_fun)

    is_finite = np.sum(np.isfinite(nll_map.flatten()))
    print(f'    {nll_fun.__name__}  got {is_finite} finite values ({100*is_finite/np.prod(nll_map.shape)} %)')

    # plot the values for which it is numerically stable:
    # set the color axis as logarithmic
    print('    plotting the objective function in parameter space...')
    z = nll_map
    zlog = np.log(z)
    (z_min, z_max) = (zlog[np.isfinite(zlog)].min(), zlog[np.isfinite(zlog)].max())
    zax   = np.linspace(z_min, z_max, 5)
    zax_t = np.exp(zax)

    fig.add_trace(go.Contour(
                    z=zlog,
                    x=alpha_grid[1,:],
                    y=beta_grid[:,1],
                    colorbar={'title':"-log-likelihood"},
                    hoverongaps = False,
                    contours=dict(
                        start=1,
                        end=10,
                        size=.5
                    )),
                    row=1, col=k_fun+1)

    fig.add_scatter(x=[alpha], y = [beta], 
                    text = ["true value"], 
                    textposition="top center", 
                    mode='markers+text', 
                    marker={"color":"white"}, 
                    textfont={'color':'white'}, 
                    hoverinfo=None,
                    showlegend=False,
                    row=1, col=k_fun+1)
    fig.update_yaxes(
        title_text="alpha",
        row=1, col=k_fun+1)
    fig.update_xaxes(
        title_text="beta",
        row=1, col=k_fun+1)

fig.update_layout(template='plotly_dark')
fig.show()

### Do the brute force optimization for several alpha/beta combinations

In [ ]:
# Do the brute force optimization for several alpha/beta combinations
do_surf = False

# repeat everything for several true values
X_ref = 20
p_ref = 1
X_lot = 30
p_lot = .75

# get all possible combinations
X_vals = [20, 30, 40, 70, 100]
p_vals = [.1, .25, .5, .75, .9]
trialsPerComb = 6

# combinations
(X_grid, p_grid) = np.meshgrid(X_vals, p_vals)
X_cmb_unique = np.tile(X_grid.flatten(), trialsPerComb)
p_cmb_unique = np.tile(p_grid.flatten(), trialsPerComb)
X_lot_trials = np.tile(X_cmb_unique, trialsPerComb)
p_lot_trials = np.tile(p_cmb_unique, trialsPerComb)


alpha_true_vals = np.linspace(0.1, 1.5, 8)
beta_true_vals = np.linspace(-2, .5, 2)

(alpha_true_grid, beta_true_grid) = np.meshgrid(alpha_true_vals, beta_true_vals)
alphabeta_shape = alpha_true_grid.shape
alpha_true_grid = alpha_true_grid.flatten()
beta_true_grid = beta_true_grid.flatten()

# plot the results
for k1 in np.arange(len(alpha_true_grid)):
    print(f'{(k1+1)}/{len(alpha_true_grid)}')

    print('generating choices...')
    choices, pk, EUr, EUl = get_choices(X_ref, p_ref, X_lot_trials, p_lot_trials, alpha_true_grid[k1], beta_true_grid[k1])
    
    # Optimize (brute force)
    alpha_range = (-2, 2)
    beta_range = (-4, 4)
    steps = 100

    print('running brute force...')
    funs = (get_nll, get_nll_clean)
    fig = make_subplots(rows=1, cols=len(funs), subplot_titles=[_.__name__ for _ in funs])
    for k_fun in np.arange(len(funs)):
        nll_fun = funs[k_fun]
        alpha_grid, beta_grid, nll = optimize_brute_force(X_ref, p_ref, X_lot_trials, p_lot_trials, choices, alpha_range, beta_range, steps, get_nll_fun=nll_fun)


        (z_min, z_max) = (nll[np.isfinite(nll)].min(), nll[np.isfinite(nll)].max())

        print('plotting...')
        '''
        fig = go.Figure(layout={
            "template":"plotly_dark",
            "scene":{
            "camera":{
                "projection":{"type":"orthographic"}
                }
            }
            })
        '''
        # log axis
        z = nll
        zlog = np.log(z)
        (z_min, z_max) = (zlog[np.isfinite(zlog)].min(), zlog[np.isfinite(zlog)].max())
        zax   = np.linspace(1, 10, 40)
        zax_t = np.exp(zax)

        fig.add_trace(go.Contour(z=zlog, 
                                    x=alpha_grid[1,:], 
                                    y=beta_grid[:,1],
                                    colorbar={'title':'-log-likelihood', 
                                            "tickvals":zax, 
                                            "ticktext":zax_t},
                                    hoverongaps = False,contours=dict(start=1,end=10,size=.25)),row=1,col=k_fun+1)
        fig.add_trace(go.Scatter(x=[alpha_true_grid[k1]], y = [beta_true_grid[k1]], 
                                    text = ["true value"], 
                                    textposition="top center", 
                                    mode='markers+text',
                                    marker={"color":"white"},
                                    textfont={'color':'white'}, 
                                    hoverinfo=None, 
                                    showlegend=False),
                                    row=1,col=k_fun+1)
        fig.update_layout(xaxis_title_text='alpha',  
                    yaxis_title_text='beta')



    fig.update_layout(template="plotly_dark",
                          scene={"camera":{"projection":{"type":"orthographic"}}})

    fig.show()

        


### Do the brute force optimization for several alpha/beta combinations (one X-p combination)

In [ ]:
# Do the brute force optimization for several alpha/beta combinations (only one lottery combination, should be ill-posed)

# repeat everything for several true values
X_ref = 20
p_ref = 1
X_lot = 30
p_lot = .75

# get all possible combinations
X_vals = [40]
p_vals = [.5]
trialsPerComb = 100 # more trials, since we have less data

# combinations
(X_grid, p_grid) = np.meshgrid(X_vals, p_vals)
X_cmb_unique = np.tile(X_grid.flatten(), trialsPerComb)
p_cmb_unique = np.tile(p_grid.flatten(), trialsPerComb)
X_lot_trials = np.tile(X_cmb_unique, trialsPerComb)
p_lot_trials = np.tile(p_cmb_unique, trialsPerComb)


alpha_true_vals = np.linspace(0.1, 1.5, 8)
beta_true_vals = np.linspace(-2, .5, 2)

(alpha_true_grid, beta_true_grid) = np.meshgrid(alpha_true_vals, beta_true_vals)
alphabeta_shape = alpha_true_grid.shape
alpha_true_grid = alpha_true_grid.flatten()
beta_true_grid = beta_true_grid.flatten()

# plot the results
for k1 in np.arange(len(alpha_true_grid)):
    print(f'{(k1+1)}/{len(alpha_true_grid)}')

    print('generating choices...')
    choices, pk, EUr, EUl = get_choices(X_ref, p_ref, X_lot_trials, p_lot_trials, alpha_true_grid[k1], beta_true_grid[k1])
    
    # Optimize (brute force)
    alpha_range = (-2, 2)
    beta_range = (-4, 4)
    steps = 100

    print('running brute force...')
    funs = (get_nll, get_nll_clean)
    fig = make_subplots(rows=1, cols=len(funs), subplot_titles=[_.__name__ for _ in funs])
    for k_fun in np.arange(len(funs)):
        nll_fun = funs[k_fun]
        alpha_grid, beta_grid, nll = optimize_brute_force(X_ref, p_ref, X_lot_trials, p_lot_trials, choices, alpha_range, beta_range, steps, get_nll_fun=nll_fun)


        (z_min, z_max) = (nll[np.isfinite(nll)].min(), nll[np.isfinite(nll)].max())

        print('plotting...')
        '''
        fig = go.Figure(layout={
            "template":"plotly_dark",
            "scene":{
            "camera":{
                "projection":{"type":"orthographic"}
                }
            }
            })
        '''
        # log axis
        z = nll
        zlog = np.log(z)
        (z_min, z_max) = (zlog[np.isfinite(zlog)].min(), zlog[np.isfinite(zlog)].max())
        zax   = np.linspace(1, 10, 40)
        zax_t = np.exp(zax)

        fig.add_trace(go.Contour(z=zlog, 
                                    x=alpha_grid[1,:], 
                                    y=beta_grid[:,1],
                                    colorbar={'title':'-log-likelihood', 
                                            "tickvals":zax, 
                                            "ticktext":zax_t},
                                    hoverongaps = False,contours=dict(start=1,end=10,size=.25)),row=1,col=k_fun+1)
        fig.add_trace(go.Scatter(x=[alpha_true_grid[k1]], y = [beta_true_grid[k1]], 
                                    text = ["true value"], 
                                    textposition="top center", 
                                    mode='markers+text',
                                    marker={"color":"white"},
                                    textfont={'color':'white'}, 
                                    hoverinfo=None, 
                                    showlegend=False),
                                    row=1,col=k_fun+1)
        fig.update_layout(xaxis_title_text='alpha',  
                    yaxis_title_text='beta')



    fig.update_layout(template="plotly_dark",
                          scene={"camera":{"projection":{"type":"orthographic"}}})

    fig.show()